<a href="https://colab.research.google.com/github/rootofdata/Outlier_Detection/blob/main/Time_Synchronization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unsupervised outlier detection for Time series data using LSTM + AE
Feature : TMP,HMD,TVOC,CO,CO2,CH2O,PM10 (7 features)   
Time : 2022-04-01 ~ 2022-05-16

## 0. Setting

### 구글코랩 환경설정

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd '/content/drive/MyDrive/Outlier Detection Paper'

/content/drive/.shortcut-targets-by-id/1S9I6ATEci1ZBNpMlPkK6JfcQv0DBJcM9/Outlier Detection Paper


In [3]:
!ls

 영석	      data		 'Jun Hyeok3.ipynb'  'Main Paper Code.ipynb'
'논문 작업'  'Jun Hyeok2.ipynb'  'Jun Hyeok.ipynb'    Reference


### 필요한 라이브러리 호출

In [4]:
from glob import glob
import os

import pandas as pd
import numpy as np
import datetime
import time

import seaborn as sns
sns.set(font_scale=1.5)
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['font.family']='Malgun Gothic'  # 한글 폰트 패치.
matplotlib.rcParams['axes.unicode_minus'] = False   

import warnings
warnings.filterwarnings('ignore')   # 경고문 처리.

from sklearn.preprocessing import StandardScaler

## 1. Data Load

### User Function Definition

### Device 11

In [6]:
filst = sorted(glob('data/*.csv'))
filst

['data/device11_0401_to_0516.csv', 'data/device16_0401_to_0516.csv']

In [7]:
device11 = pd.read_csv(filst[0])
device16 = pd.read_csv(filst[1])

In [8]:
print(device11.shape)
device11.head()

(26858, 17)


,DeviceName,EntityId,TsYMD,CH2O,PM2.5,PM10,TVOC,CO2,Temperature,Humidity,Radioactivity,Sound,CO,Stink,NO2,H2S,NH3
0,Device 10000011,d369b400-144d-11ec-80c8-b1d8e607054f,2022-04-07 10:46:38,36.0,2.0,3.0,1693.0,592.0,23.49,16.35,0.03,35.0,1.06,0.03,0.01,0.0,0.0
1,Device 10000011,d369b400-144d-11ec-80c8-b1d8e607054f,2022-04-07 10:48:38,29.0,1.0,2.0,1675.0,592.0,24.72,16.11,0.00,35.0,0.98,0.03,0.01,0.0,0.0
2,Device 10000011,d369b400-144d-11ec-80c8-b1d8e607054f,2022-04-07 10:50:38,28.0,1.0,1.0,1698.0,597.0,24.73,16.05,0.00,31.0,0.81,0.03,0.01,0.0,0.0
3,Device 10000011,d369b400-144d-11ec-80c8-b1d8e607054f,2022-04-07 10:52:38,28.0,1.0,1.0,1740.0,598.0,24.74,16.02,0.03,38.0,0.96,0.03,0.01,0.0,0.0
4,Device 10000011,d369b400-144d-11ec-80c8-b1d8e607054f,2022-04-07 10:54:38,29.0,1.0,2.0,1754.0,602.0,23.10,16.02,0.05,37.0,0.77,0.03,0.01,0.0,0.0


In [9]:
device11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26858 entries, 0 to 26857
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   DeviceName     26858 non-null  object 
 1   EntityId       26858 non-null  object 
 2   TsYMD          26858 non-null  object 
 3   CH2O           26858 non-null  float64
 4   PM2.5          26858 non-null  float64
 5   PM10           26858 non-null  float64
 6   TVOC           26858 non-null  float64
 7   CO2            26858 non-null  float64
 8   Temperature    26858 non-null  float64
 9   Humidity       26858 non-null  float64
 10  Radioactivity  26858 non-null  float64
 11  Sound          26858 non-null  float64
 12  CO             26858 non-null  float64
 13  Stink          26858 non-null  float64
 14  NO2            26858 non-null  float64
 15  H2S            26858 non-null  float64
 16  NH3            26858 non-null  float64
dtypes: float64(14), object(3)
memory usage: 3.5+ MB


In [10]:
device11.describe()

,CH2O,PM2.5,PM10,TVOC,CO2,Temperature,Humidity,Radioactivity,Sound,CO,Stink,NO2,H2S,NH3
count,26858.000000,26858.000000,26858.000000,26858.000000,26858.000000,26858.000000,26858.000000,26858.000000,26858.000000,26858.000000,26858.000000,26858.000000,26858.0,26858.0
mean,35.124618,5.687542,9.756460,1810.430412,550.039504,23.562756,19.065201,0.019528,37.923971,0.448039,0.018068,0.006507,0.0,0.0
std,43.783533,5.507792,10.235937,998.464677,124.948390,2.388771,3.654859,0.030341,4.240932,0.344297,0.014783,0.004768,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,400.000000,18.500000,9.660000,0.000000,26.000000,0.000000,0.000000,0.000000,0.0,0.0
25%,21.000000,2.000000,3.000000,1064.000000,453.000000,21.830000,16.710000,0.000000,35.000000,0.000000,0.000000,0.000000,0.0,0.0
50%,27.000000,4.000000,6.000000,1629.000000,539.000000,23.215000,18.560000,0.000000,38.000000,0.580000,0.020000,0.010000,0.0,0.0
75%,36.000000,8.000000,13.000000,2253.000000,619.000000,25.030000,20.700000,0.030000,41.000000,0.710000,0.030000,0.010000,0.0,0.0
max,926.000000,53.000000,99.000000,8440.000000,1924.000000,34.380000,33.130000,0.850000,57.000000,1.680000,0.390000,0.010000,0.0,0.0


### Device 16

In [15]:
print(device16.shape)
device16.head()

(31591, 17)


,DeviceName,EntityId,TsYMD,CH2O,PM2.5,PM10,TVOC,CO2,Temperature,Humidity,Radioactivity,Sound,CO,Stink,NO2,H2S,NH3
0,Device 10000016,08228f50-144e-11ec-80c8-b1d8e607054f,2022-04-01 08:35:52,46.0,2.0,3.0,218.0,400.0,20.53,19.86,0.03,39.0,0.39,0.43,0.01,0.0,0.0
1,Device 10000016,08228f50-144e-11ec-80c8-b1d8e607054f,2022-04-01 08:37:52,46.0,2.0,3.0,218.0,400.0,20.53,19.85,0.03,35.0,0.42,0.43,0.01,0.0,0.0
2,Device 10000016,08228f50-144e-11ec-80c8-b1d8e607054f,2022-04-01 08:39:52,46.0,2.0,3.0,215.0,400.0,19.14,19.85,0.03,34.0,0.39,0.43,0.01,0.0,0.0
3,Device 10000016,08228f50-144e-11ec-80c8-b1d8e607054f,2022-04-01 08:41:52,45.0,2.0,3.0,215.0,400.0,20.51,19.85,0.06,36.0,0.25,0.43,0.01,0.0,0.0
4,Device 10000016,08228f50-144e-11ec-80c8-b1d8e607054f,2022-04-01 08:43:52,46.0,2.0,3.0,214.0,400.0,21.89,19.84,0.03,36.0,0.35,0.43,0.01,0.0,0.0


In [16]:
device16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31591 entries, 0 to 31590
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   DeviceName     31591 non-null  object 
 1   EntityId       31591 non-null  object 
 2   TsYMD          31591 non-null  object 
 3   CH2O           31591 non-null  float64
 4   PM2.5          31591 non-null  float64
 5   PM10           31591 non-null  float64
 6   TVOC           31591 non-null  float64
 7   CO2            31591 non-null  float64
 8   Temperature    31591 non-null  float64
 9   Humidity       31591 non-null  float64
 10  Radioactivity  31591 non-null  float64
 11  Sound          31591 non-null  float64
 12  CO             31591 non-null  float64
 13  Stink          31591 non-null  float64
 14  NO2            31591 non-null  float64
 15  H2S            31591 non-null  float64
 16  NH3            31591 non-null  float64
dtypes: float64(14), object(3)
memory usage: 4.1+ MB


In [17]:
device16.describe()

,CH2O,PM2.5,PM10,TVOC,CO2,Temperature,Humidity,Radioactivity,Sound,CO,Stink,NO2,H2S,NH3
count,31591.000000,31591.000000,31591.000000,31591.000000,31591.000000,31591.000000,31591.000000,31591.000000,31591.000000,31591.000000,31591.000000,31591.000000,31591.000000,31591.000000
mean,48.285081,5.828907,9.967174,1719.719889,527.998924,23.187888,19.290544,0.015293,35.510019,0.177688,0.252847,0.005742,0.000032,0.175240
std,18.754226,5.030342,9.356082,1045.780245,116.377187,2.293207,3.450011,0.031207,4.599367,0.164155,0.222755,0.004945,0.005626,0.380261
min,8.000000,0.000000,0.000000,0.000000,400.000000,16.570000,10.890000,0.000000,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,35.000000,3.000000,5.000000,979.000000,446.000000,21.580000,16.860000,0.000000,32.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,45.000000,4.000000,7.000000,1452.000000,505.000000,22.950000,18.730000,0.000000,35.000000,0.230000,0.370000,0.010000,0.000000,0.000000
75%,60.000000,8.000000,13.000000,2182.000000,586.000000,24.600000,20.920000,0.030000,38.000000,0.310000,0.450000,0.010000,0.000000,0.000000
max,235.000000,54.000000,100.000000,8611.000000,1793.000000,33.570000,32.370000,1.870000,59.000000,0.870000,0.600000,0.010000,1.000000,2.000000


## 2. Data Prprocessing

In [22]:
d11=device11.copy()
d16=device16.copy()

### Separate device because of long term over 2min


이거 해주는 이유 바로 아래 코드 보면 이해됨.  
2022-04-13 04:27 | 2022-04-13 10:18 사이 공백 발생 -> 공백 발생하는 곳마다 여러개의 dataframe으로 나눠 진행 필요

In [23]:
d11['TsYMD'][4127:4131]
#d16['TsYMD'][8509:8513]

4127    2022-04-13 04:25:18
4128    2022-04-13 04:27:18
4129    2022-04-13 10:18:43
4130    2022-04-13 10:20:46
Name: TsYMD, dtype: object

여기부터 진행

In [24]:
d11['TsYMD']=pd.to_datetime(d11['TsYMD'])
d11=d11.sort_values('TsYMD')
d11['TsYMD']=d11['TsYMD'].dt.to_period(freq='min')
time =d11['TsYMD'].apply(lambda x: x.strftime('%Y%m%d%H%M'))

In [25]:
from datetime import datetime
check_list=[0]
for i in range(1,len(time)):
  now=datetime.strptime(time[i],'%Y%m%d%H%M')
  past=datetime.strptime(time[i-1],'%Y%m%d%H%M')
  diff=now-past
  diff=diff.seconds/60
  if diff >6:
    check_list.append(i)
check_list.append(len(time))

In [26]:
check_list

[0, 4129, 5677, 8517, 26858]

In [27]:
# check 한 번 해주기.
for i in range(len(check_list)):
  print(d11['TsYMD'][check_list[i]-1:check_list[i]+1])

Series([], Name: TsYMD, dtype: period[T])
4128    2022-04-13 04:27
4129    2022-04-13 10:18
Name: TsYMD, dtype: period[T]
5676    2022-04-15 14:29
5677    2022-04-15 14:57
Name: TsYMD, dtype: period[T]
8516    2022-04-19 15:08
8517    2022-04-20 12:00
Name: TsYMD, dtype: period[T]
26857    2022-05-15 23:59
Name: TsYMD, dtype: period[T]


In [28]:
d11_1=d11[check_list[0]:check_list[1]]
d11_2=d11[check_list[1]+1:check_list[2]]
d11_3=d11[check_list[2]+1:check_list[3]]
d11_4=d11[check_list[3]+1:check_list[4]]

device11 -> d11_1, d11_2, d11_3, d11_4로 나눠 진행

In [29]:
d11_4

,DeviceName,EntityId,TsYMD,CH2O,PM2.5,PM10,TVOC,CO2,Temperature,Humidity,Radioactivity,Sound,CO,Stink,NO2,H2S,NH3
8518,Device 10000011,d369b400-144d-11ec-80c8-b1d8e607054f,2022-04-20 12:02,24.0,22.0,41.0,17.0,401.0,22.95,16.66,0.00,34.0,0.86,0.36,0.01,0.0,0.0
8519,Device 10000011,d369b400-144d-11ec-80c8-b1d8e607054f,2022-04-20 12:04,17.0,27.0,50.0,80.0,412.0,23.30,16.48,0.04,35.0,0.71,0.33,0.01,0.0,0.0
8520,Device 10000011,d369b400-144d-11ec-80c8-b1d8e607054f,2022-04-20 12:07,17.0,24.0,44.0,115.0,410.0,23.57,16.39,0.04,38.0,0.64,0.31,0.01,0.0,0.0
8521,Device 10000011,d369b400-144d-11ec-80c8-b1d8e607054f,2022-04-20 12:08,15.0,25.0,45.0,68.0,400.0,23.36,16.16,0.02,34.0,0.77,0.28,0.01,0.0,0.0
8522,Device 10000011,d369b400-144d-11ec-80c8-b1d8e607054f,2022-04-20 12:10,15.0,22.0,41.0,90.0,401.0,23.56,16.08,0.03,40.0,0.69,0.27,0.01,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26853,Device 10000011,d369b400-144d-11ec-80c8-b1d8e607054f,2022-05-15 23:51,22.0,1.0,1.0,1325.0,403.0,21.81,16.57,0.00,42.0,0.69,0.03,0.01,0.0,0.0
26854,Device 10000011,d369b400-144d-11ec-80c8-b1d8e607054f,2022-05-15 23:53,22.0,1.0,1.0,1308.0,407.0,21.80,16.58,0.00,42.0,0.51,0.03,0.01,0.0,0.0
26855,Device 10000011,d369b400-144d-11ec-80c8-b1d8e607054f,2022-05-15 23:55,22.0,1.0,1.0,1306.0,406.0,21.81,16.58,0.00,40.0,0.67,0.03,0.01,0.0,0.0
26856,Device 10000011,d369b400-144d-11ec-80c8-b1d8e607054f,2022-05-15 23:57,22.0,1.0,1.0,1322.0,407.0,24.55,16.57,0.00,37.0,0.70,0.03,0.01,0.0,0.0


### Time Synchronization (2 min)

In [50]:
import datetime

def deviceEDA(device):
  device.reset_index(drop=True,inplace=True)
  time =device['TsYMD'].apply(lambda x: x.strftime('%Y%m%d%H%M'))
  time.reset_index(drop=True,inplace=True)
  for i in range(len(device['TsYMD'])):
    if int(time[i][-2:])%2 ==1:
      device['TsYMD'][i]=device['TsYMD'][i]+datetime.timedelta(minutes=1)
  return device

def frameEDA(device):
  frame = pd.date_range(start = device['TsYMD'].iloc[0].strftime('%Y-%m-%d %H:%M'),            # 날짜 범위 시작
                     end = device['TsYMD'].iloc[-1].strftime('%Y-%m-%d %H:%M'),                    # 날짜 범위 끝
                     freq = '2min',                           # 시간 간격( 2분 간격)
                     tz = 'Asia/Seoul')                       # 시간대(timezone)
  frame=pd.DataFrame(frame)
  frame.columns=['TsYMD']
  frame['TsYMD'] = frame['TsYMD'].dt.to_period(freq = 'min')  #분까지 끊기
  return frame

def mergeEDA(device,frame):
  merge_device = pd.merge(frame, device, how='outer',on='TsYMD')
  merge_device = merge_device.sort_values('TsYMD')
  merge_device.interpolate(method="ffill", inplace=True)
  return merge_device

def synchronization(device):
  new_device=deviceEDA(device)
  new_device_frame=frameEDA(new_device)
  new_device= mergeEDA(new_device,new_device_frame)
  return new_device

In [51]:
newd11_1=synchronization(d11_1)
newd11_2=synchronization(d11_2)
newd11_3=synchronization(d11_3)
newd11_4=synchronization(d11_4)

In [58]:
print('-----------------------')
print(len(d11_1))
print(len(newd11_1))
print('-----------------------')
print(len(d11_2))
print(len(newd11_2))
print('-----------------------')
print(len(d11_3))
print(len(newd11_3))
print('-----------------------')
print(len(d11_4))
print(len(newd11_4))
print('-----------------------')

-----------------------
4129
4134
-----------------------
1547
1567
-----------------------
2839
2889
-----------------------
18340
18370
-----------------------
